In [1]:
import datetime
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
import random
from operator import itemgetter
import zipfile
from sklearn.metrics import roc_auc_score
import time
import os
import nltk.corpus
from nltk import SnowballStemmer
import random as rnd, re
from gensim.models import Word2Vec
import editdistance
import json
import pickle
random.seed(2016)
os.chdir('/home/valesco/Datasets/Avito/')

In [2]:
testing = 0
start_time = time.time()

types1 = {
    'itemID_1': np.dtype(int),
    'itemID_2': np.dtype(int),
    'isDuplicate': np.dtype(int),
    'generationMethod': np.dtype(int),
}

types2 = {
    'itemID': np.dtype(int),
    'categoryID': np.dtype(int),
    'title': np.dtype(str),
    'description': np.dtype(str),
    'images_array': np.dtype(str),
    'attrsJSON': np.dtype(str),
    'price': np.dtype(float),
    'locationID': np.dtype(int),
    'metroID': np.dtype(float),
    'lat': np.dtype(float),
    'lon': np.dtype(float),
}

print("Load ItemPairs_train.csv")
pairs = pd.read_csv("ItemPairs_train.csv", dtype=types1)
# Add 'id' column for easy merge
print("Load ItemInfo_train.csv")
items = pd.read_csv("ItemInfo_train.csv", dtype=types2)
items.fillna(-1, inplace=True)

Load ItemPairs_train.csv
Load ItemInfo_train.csv


In [3]:
location = pd.read_csv("Location.csv")
category = pd.read_csv("Category.csv")

stopw_r = [word for word in nltk.corpus.stopwords.words("russian") if word!="не"]
stopw_e = [word for word in nltk.corpus.stopwords.words("english")]
stopw = stopw_r + stopw_e
stem_r = SnowballStemmer('russian')
stem_e = SnowballStemmer('english')

def str_stem(s):
    s = re.sub(u'[^a-zа-я0-9]', ' ', str(s).lower())
    s = (' ').join([stem_r.stem(w) if not re.search("[0-9a-z]", w) else stem_e.stem(w) for w in s.split() if len(w)>1 and w not in stopw])
    return s

start = time.time()

train = pairs
train = train.drop(['generationMethod'], axis=1)

items = items[:100]
items['len_title'] = items['title'].str.len()
items['num_words_title'] = items['title'].map(lambda x: len(str(x).split()))
items['len_description'] = items['description'].str.len()
items['num_words_description'] = items['description'].map(lambda x: len(str(x).split()))
items['len_attrsJSON'] = items['attrsJSON'].str.len()
items['title'] = items['title'].map(lambda x: str_stem(x))
items['description'] = items['description'].map(lambda x: str_stem(x))

finish = time.time()
print((finish - start)/60)

0.005963265895843506


In [19]:
items.head()



#item1 = items[['itemID', 'categoryID', 'price', 'locationID', 'metroID', 'lat', 'lon', 
    #'len_title', 'len_description', 'len_attrsJSON']]


,itemID,categoryID,title,description,images_array,attrsJSON,price,locationID,metroID,lat,lon,len_title,num_words_title,len_description,num_words_description,len_attrsJSON
0,1,81,прод камаз 6520,прод камаз 6520 20 тон,"1064094, 5252822, 6645873, 6960145, 9230265","{""Вид техники"":""Грузовики""}",300000.0,648140,-1.0,64.686946,30.815924,17,3,25,5,27.0
1,3,14,yamaha r6,ве тюнинг,"11919573, 14412228, 3204180, 6646877","{""Вид техники"":""Мотоциклы"", ""Вид мотоцикла"":""С...",300000.0,639040,-1.0,55.678037,37.256548,9,2,15,3,57.0
2,4,84,iphon 3gs 8gb,телефон хорош состоян трещин скол врем меня ак...,"14384831, 6102021","{""Вид телефона"":""iPhone""}",3500.0,640650,-1.0,56.239398,43.460458,14,3,239,35,25.0
3,7,84,xiaomi mi4 3гб ram 16гб rom бел,отличн подарок нов год китайск appl нов упаков...,-1,"{""Вид телефона"":""Другие марки""}",13500.0,662210,-1.0,55.777170,37.586194,35,8,801,114,31.0
4,8,39,лыжн ботинк,лыжн ботинк хорош состоян 34 размер,"13718854, 4787310","{""Вид товара"":""Зимние виды спорта""}",500.0,624360,-1.0,55.777170,37.586194,14,2,45,7,35.0


In [4]:
model = Word2Vec.load_word2vec_format('ruscorpora.model.bin', binary = True)

In [9]:
def similarity(bow1, bow2):
    count1 = 0
    count2 = 0
    for word1 in bow1:
        for word2 in bow2:
            try:
                temp = model.similarity(word1, word2)
                if temp > .30 or word1 == word2:
                    count1 += 1
            except:
                count2 += 1
                pass
    
    return count1/(len(bow1))
              
            

items['desc_sim'] = items['description'].map(lambda x: (similarity(x.split(' '), str(items['description'][2]).split(' '))))

In [10]:
items.sort_values(by = 'desc_sim', ascending = False)

,itemID,categoryID,title,description,images_array,attrsJSON,price,locationID,metroID,lat,lon,len_title,num_words_title,len_description,num_words_description,len_attrsJSON,desc_sim
2,4,84,iphon 3gs 8gb,телефон хорош состоян трещин скол врем меня ак...,"14384831, 6102021","{""Вид телефона"":""iPhone""}",3500.0,640650,-1.0,56.239398,43.460458,14,3,239,35,25.0,0.666667
89,152,84,iphon,полн комплект состоян нов использован месяц об...,861579,"{""Вид телефона"":""iPhone""}",34000.0,628970,-1.0,52.276547,104.284891,8,2,119,18,25.0,0.230769
37,71,10,блок управлен бу неоригина опел,блок управлен бу неоригина опел,-1,"{""Вид товара"":""Запчасти"", ""Тип товара"":""Для ав...",1900.0,637640,-1.0,55.753653,37.619800,38,6,38,6,95.0,0.200000
44,80,112,сиделк,ищ работ сиделк оп работ больш напарниц опыт р...,-1,"{""Образование"":""Среднее"", ""Пол"":""Женский"", ""Во...",-1.0,633270,-1.0,55.777170,37.586194,7,1,121,15,244.0,0.142857
58,101,10,усилител довесок самбуфер,xstream xta 360 довесок отд самбуфер провод ра...,"10940316, 5744817, 6352755","{""Вид товара"":""Аудио- и видеотехника""}",4000.0,637640,174.0,55.692440,37.534532,30,5,161,24,38.0,0.117647
50,89,31,mac mini обм ноутбук айфон,магазин состоян купл связн чек гарант коробк п...,"4696617, 6269860, 6360062, 951825",-1,42000.0,633080,-1.0,44.585592,38.055061,35,7,171,13,NaN,0.100000
88,149,30,машинк peppa pig свинк пепп,набор вход фигурк мам пап пепп джордж фигурок ...,"2612626, 441240, 4954671, 5071736, 648327","{""Вид товара"":""Игрушки""}",1399.0,637640,500645.0,55.794376,37.799364,31,5,259,40,24.0,0.096774
49,87,30,прод коляск,прод итальянск коляск cam состоян хорош одн ре...,"12029119, 2081990, 5200820, 6240610, 7000448","{""Вид товара"":""Детские коляски""}",10000.0,625810,-1.0,55.777170,37.586194,14,2,390,49,32.0,0.095238
45,81,90,британск котят,прод британск прямоух кот лилов голуб британц ...,"11169301, 13257007, 1755839, 6129504, 9370081","{""Порода"":""Британская""}",4500.0,637640,500033.0,55.887473,37.661527,17,2,299,45,23.0,0.090909
54,95,84,iphon 5c 16gb,phone 5c 16gb 10000 торг купл июн виде 16 999 ...,"12625416, 2547493","{""Вид телефона"":""iPhone""}",9500.0,637640,-1.0,55.753653,37.619800,14,3,861,131,25.0,0.074074


In [34]:
model.similarity('телефон','телефон')

1.0

In [59]:
(items['description'][6]).split(' ')

['машин',
 'нов',
 'пробег',
 'реальн',
 'не',
 'бит',
 'не',
 'крашен',
 'одн',
 'хозяин',
 'комплектац',
 'максимальн',
 'полн',
 'люкс',
 'номер',
 'подарок']

In [2]:
types1 = {
        'itemID_1': np.dtype(int),
        'itemID_2': np.dtype(int),
        'id': np.dtype(int),
    }

types2 = {
    'itemID': np.dtype(int),
    'categoryID': np.dtype(int),
    'title': np.dtype(str),
    'description': np.dtype(str),
    'images_array': np.dtype(str),
    'attrsJSON': np.dtype(str),
    'price': np.dtype(float),
    'locationID': np.dtype(int),
    'metroID': np.dtype(float),
    'lat': np.dtype(float),
    'lon': np.dtype(float),
}

print("Load ItemPairs_train.csv")
pairs = pd.read_csv("ItemPairs_train.csv", dtype=types1)
print("Load ItemInfo_train.csv")
train = pd.read_csv("ItemInfo_train.csv", dtype=types2)
train.fillna(-1, inplace=True)
location = pd.read_csv("Location.csv")
category = pd.read_csv("Category.csv")

Load ItemPairs_train.csv
Load ItemInfo_train.csv


In [3]:
pairs = pairs[:100]
pairs.rename(columns = {'itemID': 'itemID_1'}, inplace = True)
train.rename(columns = {'itemID': 'itemID_1'}, inplace = True)
train_df = pd.merge(pairs, train, how = 'left', on = 'itemID_1', left_index = True)

item1 = train

In [4]:
item1 = item1.rename(
        columns={
            'itemID': 'itemID_1',
            'categoryID': 'categoryID_1',
            'parentCategoryID': 'parentCategoryID_1',
            'price': 'price_1',
            'locationID': 'locationID_1',
            'regionID': 'regionID_1',
            'metroID': 'metroID_1',
            'lat': 'lat_1',
            'lon': 'lon_1',
            'title': 'title_1',
            'description': 'desc_1',
            'attrsJSON': 'JSON_1',
            'images_array': 'images_array_1'
            })

In [5]:
item2 = train
item2 = item2.rename(
        columns={
            'itemID_1': 'itemID_2',
            'categoryID': 'categoryID_2',
            'parentCategoryID': 'parentCategoryID_2',
            'price': 'price_2',
            'locationID': 'locationID_2',
            'regionID': 'regionID_2',
            'metroID': 'metroID_2',
            'lat': 'lat_2',
            'lon': 'lon_2',
            'len_title': 'len_title_2',
            'title': 'title_2',
            'description': 'desc_2',
            'attrsJSON': 'JSON_2',
            'images_array': 'images_array_2'
            })

In [6]:
train_df = pd.merge(pairs, item1, how = 'left', on = 'itemID_1', left_index = True)

In [7]:
train_df = pd.merge(train_df, item2, how = 'left', on = 'itemID_2', left_index = True)


In [8]:
attrib = train_df[['itemID_1', 'itemID_2', 'isDuplicate', 'categoryID_1','title_1','desc_1',
                  'JSON_1','categoryID_2', 'title_2','desc_2','JSON_2']]

In [9]:
attrib.reset_index(inplace = True)
attrib.head(20)

,index,itemID_1,itemID_2,isDuplicate,categoryID_1,title_1,desc_1,JSON_1,categoryID_2,title_2,desc_2,JSON_2
0,2255671,1,4112648,1,81,Продам Камаз 6520,Продам Камаз 6520 20 тонн,"{""Вид техники"":""Грузовики""}",81,Продам Камаз 6520,Продам Камаз 6520 20 тонн,"{""Вид техники"":""Грузовики""}"
1,1095639,3,1991275,1,14,Yamaha r6,Весь в тюнинге.,"{""Вид техники"":""Мотоциклы"", ""Вид мотоцикла"":""С...",14,Yamaha R6,Весь в тюнинге. Возможен торг,"{""Вид техники"":""Мотоциклы"", ""Вид мотоцикла"":""С..."
2,674399,4,1223296,0,84,iPhone 3gs 8gb,"Телефон в хорошем состоянии, трещин и сколов н...","{""Вид телефона"":""iPhone""}",84,iPhone 3gs 32gb,"В отличном состоянии, в комплекте зарядник и к...","{""Вид телефона"":""iPhone""}"
3,584073,7,1058851,1,84,Xiaomi Mi4 3гб RAM + 16гб ROM белый,"Отличный подарок на новый год от ""китайской ap...","{""Вид телефона"":""Другие марки""}",84,Смартфон Xiaomi Mi4 3гб RAM + 16гб ROM белый,"Отличный подарок на новый год от ""китайской ap...","{""Вид телефона"":""Другие марки""}"
4,1188720,8,2161930,1,39,Лыжные ботинки,"Лыжные ботинки в хорошем состоянии, 34 размер","{""Вид товара"":""Зимние виды спорта""}",39,Ботинки для лыж,"Ботинки 34 размер, в хорошем состоянии","{""Вид товара"":""Зимние виды спорта""}"
5,383022,9,694103,1,39,Сноуборд ботинки Nitro Team 10 us,"сноубордические ботинки Nitro Team\nразмер 42,...","{""Вид товара"":""Зимние виды спорта""}",39,Сноубордические ботинки,"ботинки Nitro Team\nразмер 42,5\n28см, 10 us\n...","{""Вид товара"":""Зимние виды спорта""}"
6,3086345,12,5637025,0,9,"LADA Priora, 2015",Машина новая пробег реальный. Не битая не краш...,"{""Марка"":""ВАЗ (LADA)"", ""Модель"":""Priora"", ""Тип...",9,"LADA Priora, 2015",Машина как новая. Комплектация максимальная- п...,"{""Марка"":""ВАЗ (LADA)"", ""Модель"":""Priora"", ""Тип..."
7,2891680,12,5279740,0,9,"LADA Priora, 2015",Машина новая пробег реальный. Не битая не краш...,"{""Марка"":""ВАЗ (LADA)"", ""Модель"":""Priora"", ""Тип...",9,"LADA Priora, 2015",Машина в идеальном состояний. Не битая не краш...,"{""Марка"":""ВАЗ (LADA)"", ""Модель"":""Priora"", ""Тип..."
8,62794,15,113701,0,32,Телевизор,"Телевизоры кинескопные, диагональ от 37 см. до...","{""Вид товара"":""Телевизоры и проекторы""}",32,Panasonic 72 см. 100Hz,Кинескопный. Отличное состояние. Великолепное ...,"{""Вид товара"":""Телевизоры и проекторы""}"
9,1356422,16,2467698,0,27,Шуба мутоновая,"качественная, производство Казань, хорошее сос...","{""Вид одежды"":""Женская одежда"", ""Предмет одежд...",27,Шуба норковая,"в отличном состоянии, можно мерить с 48 по 52 ...","{""Вид одежды"":""Женская одежда"", ""Предмет одежд..."


In [10]:
title1 = attrib.iloc[5]['title_1']
title2 = attrib.iloc[5]['title_2']

print('Title 1: {} '.format(title1))
print('Title 2: {} '.format(title2))

Title 1: Сноуборд ботинки Nitro Team 10 us 
Title 2: Сноубордические ботинки 


In [59]:
split1 = title1.split(' ')
split2 = title2.split(' ')

edit_whole = editdistance.eval(title1, title2)

print('Entire tile edit distance: {}'.format(edit_whole))

for s1 in split1:
    for s2 in split2:
        ed = editdistance.eval(s1, s2)
        if ed < 2:
            print('Similar Words EditDist: {} ,String1: {} , String2: {}'.format(ed, s1, s2))
        else:
            print('Different Words EditDist: {} ,String1: {} , String2: {}'.format(ed, s1, s2))
            

Entire tile edit distance: 22
Different Words EditDist: 7 ,Word1: Сноуборд , Word2: Сноубордические
Different Words EditDist: 7 ,Word1: Сноуборд , Word2: ботинки
Different Words EditDist: 10 ,Word1: ботинки , Word2: Сноубордические
Similar Words EditDist: 0 ,Word1: ботинки , Word2: ботинки
Different Words EditDist: 15 ,Word1: Nitro , Word2: Сноубордические
Different Words EditDist: 7 ,Word1: Nitro , Word2: ботинки
Different Words EditDist: 15 ,Word1: Team , Word2: Сноубордические
Different Words EditDist: 7 ,Word1: Team , Word2: ботинки
Different Words EditDist: 15 ,Word1: 10 , Word2: Сноубордические
Different Words EditDist: 7 ,Word1: 10 , Word2: ботинки
Different Words EditDist: 15 ,Word1: us , Word2: Сноубордические
Different Words EditDist: 7 ,Word1: us , Word2: ботинки


In [66]:
string1 = 'Сноуборд'
string2 = 'Сноубордические'

for i in range(len(string2)):
    temp = editdistance.eval(string1, string2[0:i])
    print('{} , {}, ed = {}'.format(string1, string2[0:i], temp))



Сноуборд , , ed = 8
Сноуборд , С, ed = 7
Сноуборд , Сн, ed = 6
Сноуборд , Сно, ed = 5
Сноуборд , Сноу, ed = 4
Сноуборд , Сноуб, ed = 3
Сноуборд , Сноубо, ed = 2
Сноуборд , Сноубор, ed = 1
Сноуборд , Сноуборд, ed = 0
Сноуборд , Сноуборди, ed = 1
Сноуборд , Сноубордич, ed = 2
Сноуборд , Сноубордиче, ed = 3
Сноуборд , Сноубордичес, ed = 4
Сноуборд , Сноубордическ, ed = 5
Сноуборд , Сноубордически, ed = 6


In [9]:
title1 = attrib.iloc[16]['desc_1']
title2 = attrib.iloc[16]['desc_2']

title1 = 'xiaomi mi4 3гб ram 16гб rom бел'
title2 = 'смартфон xiaomi mi4 3гб ram 16гб rom бел'

split1 = title1.split(' ')
split2 = title2.split(' ')

print('Desc 1:')
print(title1, '\n')
print('Desc_2:')
print(title2, '\n')
no = 0
yes = 0
for s1 in split1:
    if s1 in title2:
        yes += 1
    else:
        no += 1
try:
    print('String 1 Yes/No ratio: ', yes/no)
except ZeroDivisionError:
    print('Every Word in string 5')
        
yes = 0 
no = 0
for s2 in split2:
    if s2 in title1:
        yes += 1
    else:
        no += 1
    
try:
    print('String 2 Yes/No ratio: ', yes/no)      
except ZeroDivisionError:
    print('Every Word in string 5')

Desc 1:
xiaomi mi4 3гб ram 16гб rom бел 

Desc_2:
смартфон xiaomi mi4 3гб ram 16гб rom бел 

Every Word in string 5
String 2 Yes/No ratio:  7.0


In [29]:
json1 = attrib.iloc[4]['JSON_1']
json2 = attrib.iloc[4]['JSON_2']

json1_data = json.loads(json1)
json2_data = json.loads(json2)

json1_keys = json1_data.keys()
json2_keys = json2_data.keys()

for key in json1_keys:
    if key in json2_keys:
        
    



Зимние виды спорта ,  Зимние виды спорта


In [11]:
dhash = pickle.load(open('dhash.pkl', 'rb'))
whash = pickle.load(open('whash_haar.pkl', 'rb'))

In [12]:
def hDist(s1, s2):
    s1, s2 = str(s1), str(s2)
    return sum(bool(ord(ch1) - ord(ch2)) for ch1, ch2 in zip(s1, s2))

def compare_hashes(df):
    if df['images_array_1'] == -1 or df['images_array_2'] == -1:
        hash_comp = -1
    else:
        hash_array1 = (str(df['images_array_1'])).replace(' ','')
        hash_array2 = (str(df['images_array_2'])).replace(' ','')

        hash_ls1 = hash_array1.split(',')
        hash_ls2 = hash_array2.split(',')

        num_images = len(hash_ls1)
        if len(hash_ls2) < num_images:
            num_images = len(hash_ls2)

        count = 0
        bound = 999
        hash_ls = []
        dist_ls = []
        for h1 in hash_ls1:
            if int(h1) in whash:
                for h2 in hash_ls2:
                    if int(h2) in whash:
                        dist = hDist(whash[int(h1)], whash[int(h2)])
                        if dist < bound:
                            dist_ls.append(dist)
                            
        hash_comp = count/num_images
                        
    return dist_ls
            
compare_hashes(train_df.ix[1095639])

[10, 10, 11, 0, 10, 14, 13, 14, 11, 14, 15, 16, 13, 10, 0, 0, 16, 9, 10, 15]

In [14]:
lsit = []
for i in range(10):
    lsit.append(i)
    
iteration = 0
l = len(lsit)

while iteration < l:
    print(lsit[iteration:(iteration+2)])
    iteration += 2

[0, 1]
[2, 3]
[4, 5]
[6, 7]
[8, 9]


In [61]:
train_df.head(20)

,itemID_1,itemID_2,isDuplicate,generationMethod,categoryID_1,title_1,desc_1,images_array_1,JSON_1,price_1,...,categoryID_2,title_2,desc_2,images_array_2,JSON_2,price_2,locationID_2,metroID_2,lat_2,lon_2
2255671,1,4112648,1,1,81,Продам Камаз 6520,Продам Камаз 6520 20 тонн,"1064094, 5252822, 6645873, 6960145, 9230265","{""Вид техники"":""Грузовики""}",300000.0,...,81,Продам Камаз 6520,Продам Камаз 6520 20 тонн,"1227519, 1374615, 7072137, 8671835","{""Вид техники"":""Грузовики""}",300000.0,648140,-1.0,64.686946,30.815924
1095639,3,1991275,1,1,14,Yamaha r6,Весь в тюнинге.,"11919573, 14412228, 3204180, 6646877","{""Вид техники"":""Мотоциклы"", ""Вид мотоцикла"":""С...",300000.0,...,14,Yamaha R6,Весь в тюнинге. Возможен торг,"11068709, 13325040, 13783238, 206652, 9458537","{""Вид техники"":""Мотоциклы"", ""Вид мотоцикла"":""С...",330000.0,639040,-1.0,55.678037,37.256548
674399,4,1223296,0,1,84,iPhone 3gs 8gb,"Телефон в хорошем состоянии, трещин и сколов н...","14384831, 6102021","{""Вид телефона"":""iPhone""}",3500.0,...,84,iPhone 3gs 32gb,"В отличном состоянии, в комплекте зарядник и к...",5709245,"{""Вид телефона"":""iPhone""}",3500.0,640650,-1.0,56.239398,43.460458
584073,7,1058851,1,1,84,Xiaomi Mi4 3гб RAM + 16гб ROM белый,"Отличный подарок на новый год от ""китайской ap...",-1,"{""Вид телефона"":""Другие марки""}",13500.0,...,84,Смартфон Xiaomi Mi4 3гб RAM + 16гб ROM белый,"Отличный подарок на новый год от ""китайской ap...",7900519,"{""Вид телефона"":""Другие марки""}",13500.0,662210,-1.0,56.135459,47.235484
1188720,8,2161930,1,1,39,Лыжные ботинки,"Лыжные ботинки в хорошем состоянии, 34 размер","13718854, 4787310","{""Вид товара"":""Зимние виды спорта""}",500.0,...,39,Ботинки для лыж,"Ботинки 34 размер, в хорошем состоянии","10053682, 986143","{""Вид товара"":""Зимние виды спорта""}",600.0,624360,-1.0,55.777170,37.586194
383022,9,694103,1,1,39,Сноуборд ботинки Nitro Team 10 us,"сноубордические ботинки Nitro Team\nразмер 42,...","12418395, 9930491","{""Вид товара"":""Зимние виды спорта""}",7000.0,...,39,Сноубордические ботинки,"ботинки Nitro Team\nразмер 42,5\n28см, 10 us\n...","4376062, 6606867","{""Вид товара"":""Зимние виды спорта""}",7000.0,644200,-1.0,58.004785,56.237654
3086345,12,5637025,0,1,9,"LADA Priora, 2015",Машина новая пробег реальный. Не битая не краш...,"1338189, 1648456, 6321889, 9883716","{""Марка"":""ВАЗ (LADA)"", ""Модель"":""Priora"", ""Тип...",445000.0,...,9,"LADA Priora, 2015",Машина как новая. Комплектация максимальная- п...,"10962609, 11159735, 12700029, 2565407, 4141449","{""Марка"":""ВАЗ (LADA)"", ""Модель"":""Priora"", ""Тип...",450000.0,631060,-1.0,44.219841,42.058825
2891680,12,5279740,0,1,9,"LADA Priora, 2015",Машина новая пробег реальный. Не битая не краш...,"1338189, 1648456, 6321889, 9883716","{""Марка"":""ВАЗ (LADA)"", ""Модель"":""Priora"", ""Тип...",445000.0,...,9,"LADA Priora, 2015",Машина в идеальном состояний. Не битая не краш...,"10386639, 1121974, 12223908, 1694649, 299027, ...","{""Марка"":""ВАЗ (LADA)"", ""Модель"":""Priora"", ""Тип...",455000.0,631060,-1.0,44.219841,42.058825
62794,15,113701,0,1,32,Телевизор,"Телевизоры кинескопные, диагональ от 37 см. до...","11244051, 14467554, 2240467, 5099565, 8002433","{""Вид товара"":""Телевизоры и проекторы""}",1600.0,...,32,Panasonic 72 см. 100Hz,Кинескопный. Отличное состояние. Великолепное ...,"11354553, 11364107, 13290240, 2896427, 3489831","{""Вид товара"":""Телевизоры и проекторы""}",5000.0,662810,-1.0,57.622434,39.887894
1356422,16,2467698,0,1,27,Шуба мутоновая,"качественная, производство Казань, хорошее сос...","11762574, 316289, 4015142","{""Вид одежды"":""Женская одежда"", ""Предмет одежд...",1000.0,...,27,Шуба норковая,"в отличном состоянии, можно мерить с 48 по 52 ...","12038981, 188942, 69387","{""Вид одежды"":""Женская одежда"", ""Предмет одежд...",7500.0,657600,-1.0,56.495116,84.972128


In [56]:
for k, v in dhash.items():
    if int(k) == 11919573:
        print(k)

11919573
